In [ ]:
import rasterio
from rasterio.features import shapes
mask = None
with rasterio.Env():
    with rasterio.open('C://Users/USER/Desktop/Master_Irrigation/03_GIS/irrigation_map_whr/irrigation_map_whr_georef.tif') as src:
        image = src.read(3) # first band
        results = (
        {'properties': {'raster_val': v}, 'geometry': s}
        for i, (s, v) 
        in enumerate(
            shapes(image, mask=mask, transform=src.transform)))

In [ ]:
geoms = list(results)
import geopandas as gp
gpd_polygonized_raster  = gp.GeoDataFrame.from_features(geoms)

In [ ]:
gpd_polygonized_raster.to_file('C://Users/USER/Desktop/Master_Irrigation/03_GIS/irrigation_map_whr/shp/irrigation_map_whr_georef_3.shp')

In [ ]:
import numpy as np
from skimage import io, morphology, measure
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt

img = io.imread('C://Users/USER/Desktop/Master_Irrigation/03_GIS/irrigation_map_whr/irrigation_map_whr.tiff')

rows, cols, bands = img.shape
X = img.reshape(rows*cols, bands)

kmeans = KMeans(n_clusters=16).fit(X)
labels = kmeans.labels_.reshape(rows, cols)

for i in np.unique(labels):
    blobs = np.int_(morphology.binary_opening(labels == i))
    color = np.around(kmeans.cluster_centers_[i])
    count = len(np.unique(measure.label(blobs))) - 1
    print('Color: {}  >>  Objects: {}'.format(color, count))
    
#Plot the data
plt.figure(figsize=(20,20))
plt.imshow(labels)

plt.show()

In [ ]:
from pikepdf import Pdf, PdfImage

filename = "C://Users/USER/Desktop/Master_Irrigation/03_GIS/irrigation_map_whr/irrigation_map_whr.pdf"
example = Pdf.open(filename)
image.extract_to(fileprefix="C://Users/USER/Desktop/Master_Irrigation/03_GIS/irrigation_map_whr/irrigation_map_whr-E.tif")
for i, page in enumerate(example.pages):
    for j, (name, raw_image) in enumerate(page.images.items()):
        image = PdfImage(raw_image)
        out = image.extract_to(fileprefix=f"{filename}-page{i:03}-img{j:03}")

In [ ]:
from skimage import data, io, segmentation, color
from skimage.future import graph
import numpy as np


def _weight_mean_color(graph, src, dst, n):
    """Callback to handle merging nodes by recomputing mean color.

    The method expects that the mean color of `dst` is already computed.

    Parameters
    ----------
    graph : RAG
        The graph under consideration.
    src, dst : int
        The vertices in `graph` to be merged.
    n : int
        A neighbor of `src` or `dst` or both.

    Returns
    -------
    data : dict
        A dictionary with the `"weight"` attribute set as the absolute
        difference of the mean color between node `dst` and `n`.
    """

    diff = graph.nodes[dst]['mean color'] - graph.nodes[n]['mean color']
    diff = np.linalg.norm(diff)
    return {'weight': diff}


def merge_mean_color(graph, src, dst):
    """Callback called before merging two nodes of a mean color distance graph.

    This method computes the mean color of `dst`.

    Parameters
    ----------
    graph : RAG
        The graph under consideration.
    src, dst : int
        The vertices in `graph` to be merged.
    """
    graph.nodes[dst]['total color'] += graph.nodes[src]['total color']
    graph.nodes[dst]['pixel count'] += graph.nodes[src]['pixel count']
    graph.nodes[dst]['mean color'] = (graph.nodes[dst]['total color'] /
                                      graph.nodes[dst]['pixel count'])

import rasterio


img = io.imread('C://Users/USER/Desktop/Master_Irrigation/03_GIS/irrigation_map_whr/irrigation_map_whr.tiff')
#img = data.coffee()
print(type(img), img.shape)
labels = segmentation.slic(img, compactness=30, n_segments=400, start_label=1)
g = graph.rag_mean_color(img, labels)

labels2 = graph.merge_hierarchical(labels, g, thresh=35, rag_copy=False,
                                   in_place_merge=True,
                                   merge_func=merge_mean_color,
                                   weight_func=_weight_mean_color)

out = color.label2rgb(labels2, img, kind='avg', bg_label=0)
out = segmentation.mark_boundaries(out, labels2, (0, 0, 0))
io.imshow(out)
io.show()